In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))
import torch
from numpy._core.multiarray import scalar
from models.change_classifier import ChangeClassifier

# Add numpy scalar to safe globals BEFORE loading
torch.serialization.add_safe_globals([scalar])

model = ChangeClassifier(weights=None)

# Now load with weights_only=True (safe mode)
ckpt = torch.load("../pretrained_models/checkpoint_086.pth", weights_only=False)
model.load_state_dict(ckpt["model_state_dict"])
model.eval()
torch.save(model.state_dict(), "../pretrained_models/model_entire.pth")

In [ ]:
import torchvision
from torchvision.models.efficientnet import EfficientNet

def get_exact_efficientnet_channels(model_name):
    # Create model to extract exact channel dimensions
    model = torchvision.models.__dict__[model_name](weights=None)
    
    channels = []
    for name, module in model.features.named_children():
        # For MBConv blocks, get the output channels
        if hasattr(module, 'block') and hasattr(module.block, 'out_channels'):
            channels.append(module.block.out_channels)
        # For the first conv layer
        elif name == '0' and hasattr(module, 'out_channels'):
            channels.append(module.out_channels)
    
    return channels

# Get exact channel dimensions for both models
b4_channels = get_exact_efficientnet_channels('efficientnet_b4')
b1_channels = get_exact_efficientnet_channels('efficientnet_b1')

print(f"B4 channels: {b4_channels}")
print(f"B1 channels: {b1_channels}")

# Calculate the exact scaling factors
scaling_factors = [b1 / b4 for b1, b4 in zip(b1_channels, b4_channels)]
print(f"Scaling factors: {[f'{x:.3f}' for x in scaling_factors]}")

# Get the specific layers we need (assuming layers 1, 2, 3 as in your original code)
b4_target_dims = [48, 64, 112]  # From your original MixingMaskAttentionBlock
b1_target_dims = [b1_channels[i] for i in [1, 2, 3]]  # Corresponding layers in B1

print(f"\nB4 target dimensions: {b4_target_dims}")
print(f"B1 target dimensions: {b1_target_dims}")

# Generate the MixingMaskAttentionBlock parameters
print("\nMixing blocks for B1:")
for i, dim in enumerate(b1_target_dims):
    in_channels = dim
    out_channels = dim // 2
    print(f"Layer {i+1}: MixingMaskAttentionBlock({in_channels}, {out_channels}, "
          f"[{out_channels}, {out_channels//2}, {out_channels//4}], "
          f"[{out_channels//2}, {out_channels//4}, 1])")

# Generate up_dims
print(f"\nUp dims for B1: [(2, {b1_target_dims[2]//2}, {b1_target_dims[1]}), (2, {b1_target_dims[1]}, {b1_target_dims[0]}), (2, {b1_target_dims[0]}, 32)]")

B1 dimensions: [40, 48, 80]
Mixing blocks for B1:
Layer 1: MixingMaskAttentionBlock(40, 20, [20, 10, 5], [10, 5, 1])
Layer 2: MixingMaskAttentionBlock(48, 24, [24, 12, 6], [12, 6, 1])
Layer 3: MixingMaskAttentionBlock(80, 40, [40, 20, 10], [20, 10, 1])

Up dims for B1: [(2, 40, 48), (2, 48, 40), (2, 40, 32)]


In [2]:
import sys
import os
sys.path.append(os.path.abspath(".."))
# Quick test script
from models.change_classifier import ChangeClassifier

# Test all backbones
for backbone in ["efficientnet_b4", "efficientnet_b3", "efficientnet_b2", "efficientnet_b1"]:
    try:
        model = ChangeClassifier(bkbn_name=backbone)
        param_count = sum(p.numel() for p in model.parameters())
        print(f"{backbone}: {param_count:,} parameters - ✅ Loaded successfully")
    except Exception as e:
        print(f"{backbone}: ❌ Failed to load - {e}")

efficientnet_b4: 285,803 parameters - ✅ Loaded successfully
efficientnet_b3: ❌ Failed to load - Invalid Weight class provided; expected EfficientNet_B3_Weights but received EfficientNet_B4_Weights.
efficientnet_b2: ❌ Failed to load - Invalid Weight class provided; expected EfficientNet_B2_Weights but received EfficientNet_B4_Weights.
efficientnet_b1: ❌ Failed to load - Invalid Weight class provided; expected EfficientNet_B1_Weights but received EfficientNet_B4_Weights.


In [13]:
import sys
import os
sys.path.append(os.path.abspath(".."))
import torch
import numpy as np

# Try to identify what exactly is causing the issue
from models.change_classifier import ChangeClassifier

# Create a simple test to see what's in the checkpoint
try:
    # Try to peek at the checkpoint structure without loading numpy scalars
    checkpoint = torch.load("../pretrained_models/checkpoint_086.pth", 
                           weights_only=False)
    
    print("Checkpoint keys:", checkpoint.keys())
    print("Model state dict keys sample:", list(checkpoint['model_state_dict'].keys())[:5])
    
    # Now try to load just the model safely
    model = ChangeClassifier(weights=None)
    
    # Manually filter out any problematic tensors if needed
    safe_state_dict = {}
    for key, value in checkpoint['model_state_dict'].items():
        if isinstance(value, torch.Tensor):
            safe_state_dict[key] = value
        else:
            print(f"Non-tensor found in state_dict: {key} - {type(value)}")
            # Convert non-tensors to tensors if possible
            if hasattr(value, '__array__'):
                safe_state_dict[key] = torch.tensor(np.array(value))
    
    model.load_state_dict(safe_state_dict, strict=False)
    model.eval()
    torch.save(model.state_dict(), "../pretrained_models/model_entire.pth")
    print("Success!")

except Exception as e:
    print(f"Error: {e}")

Checkpoint keys: dict_keys(['epoch', 'model_state_dict', 'optimizer_state_dict', 'scheduler_state_dict', 'loss'])
Model state dict keys sample: ['_retina.dog.weight', '_backbone.0.0.weight', '_backbone.0.1.weight', '_backbone.0.1.bias', '_backbone.0.1.running_mean']
Success!


In [14]:
import torch
from models.change_classifier import ChangeClassifier

# Load the clean model file (safe with weights_only=True)
model = ChangeClassifier(weights=None)
model.load_state_dict(torch.load("../pretrained_models/model_entire.pth", weights_only=True))


<All keys matched successfully>

In [4]:
import torch
from thop import profile, clever_format
from models.change_classifier import ChangeClassifier

# Dummy input
ref = torch.randn(1, 3, 256, 256)
test = torch.randn(1, 3, 256, 256)

# Model init
model = ChangeClassifier(
    weights=None,
    output_layer_bkbn="3",
    freeze_backbone=False
)

model.eval()

# Forward pass test
with torch.no_grad():
    out = model(ref, test)

print("✅ Forward pass OK")
print("Test shape:", test.shape)
print("Ref shape:", ref.shape)

print("Output shape:", out.shape)

# Params count
total_params = sum(p.numel() for p in model.parameters())
print(f"Total Parameters: {total_params:,}")

# FLOPs + MACs
flops, params = profile(model, inputs=(ref, test))
flops, params = clever_format([flops, params], "%.3f")
print(f"FLOPs: {flops}")
print(f"THOP Params: {params}")


✅ Forward pass OK
Test shape: torch.Size([1, 3, 256, 256])
Ref shape: torch.Size([1, 3, 256, 256])
Output shape: torch.Size([1, 1, 256, 256])
Total Parameters: 285,803
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.instancenorm.InstanceNorm2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_prelu() for <class 'torch.nn.modules.activation.PReLU'>.
[INFO] Register count_upsample() for <class 'torch.nn.modules.upsampling.Upsample'>.
FLOPs: 1.633G
THOP Params: 285.803K


In [5]:
import torch
from thop import profile, clever_format
from models.change_classifier import ChangeClassifier

ref = torch.randn(1, 3, 256, 256)
test = torch.randn(1, 3, 256, 256)

model = ChangeClassifier(
    weights=None,
    output_layer_bkbn="3",
    freeze_backbone=False
)

model.eval()
# Forward test
with torch.no_grad():
    out = model(ref, test)

print("✅ Forward pass OK")
print("Test shape:", test.shape)
print("Ref shape:", ref.shape)
print("Output shape:", out.shape)

# Param count
#total_params = sum(p.numel() for p in model.parameters())
#print(f"🧠 Total Parameters: {total_params:,}")

# FLOPs + MACs
flops, params = profile(model, inputs=(ref, test))
flops, params = clever_format([flops, params], "%.3f")
print(f"⚙️ FLOPs: {flops}")
print(f"📦 THOP Params: {params}")

# ✅ Print the full model architecture
print("🔍 Full Model Structure:\n")
print(model)
print("\n" + "="*60 + "\n")

✅ Forward pass OK
Test shape: torch.Size([1, 3, 256, 256])
Ref shape: torch.Size([1, 3, 256, 256])
Output shape: torch.Size([1, 1, 256, 256])
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.instancenorm.InstanceNorm2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_prelu() for <class 'torch.nn.modules.activation.PReLU'>.
[INFO] Register count_upsample() for <class 'torch.nn.modules.upsampling.Upsample'>.
⚙️ FLOPs: 1.633G
📦 THOP Params: 285.803K
🔍 Full Model Structure:

ChangeClassifier(
  (_retina): RetinaSimBlock(
    (dog): Conv2d(3, 3, kernel_size=(15, 15), stride=(1, 1), padding=(7, 7), groups=3, bias=False)
    (adapt): InstanceNorm2d(3, eps=1e-05, 

In [ ]:
from torchviz import make_dot
# example with dummy input (make sure inputs match your model)
t1_input = torch.randn(1, 3, 256, 256)
t2_input = torch.randn(1, 3, 256, 256)
out = model(t1_input, t2_input)
# Create graph
dot = make_dot(out, params=dict(model.named_parameters()))
# Save to file
dot.format = 'png'
dot.render('../pretrained_models/model_graph')


'model_graph.png'

In [ ]:
t1_input = torch.randn(1, 3, 256, 256)
t2_input = torch.randn(1, 3, 256, 256)
model.eval()
torch.onnx.export(
    model,
    (t1_input, t2_input),  # <--- pack as a tuple
    "../pretrained_models/model.onnx",
    opset_version=11,
    input_names=["img1", "img2"],
    output_names=["pred"]
)

In [11]:
for name, module in model.named_modules():
    print(name)


_retina
_retina.dog
_retina.adapt
_retina.act
_backbone
_backbone.0
_backbone.0.0
_backbone.0.1
_backbone.0.2
_backbone.1
_backbone.1.0
_backbone.1.0.block
_backbone.1.0.block.0
_backbone.1.0.block.0.0
_backbone.1.0.block.0.1
_backbone.1.0.block.0.2
_backbone.1.0.block.1
_backbone.1.0.block.1.avgpool
_backbone.1.0.block.1.fc1
_backbone.1.0.block.1.fc2
_backbone.1.0.block.1.activation
_backbone.1.0.block.1.scale_activation
_backbone.1.0.block.2
_backbone.1.0.block.2.0
_backbone.1.0.block.2.1
_backbone.1.0.stochastic_depth
_backbone.1.1
_backbone.1.1.block
_backbone.1.1.block.0
_backbone.1.1.block.0.0
_backbone.1.1.block.0.1
_backbone.1.1.block.0.2
_backbone.1.1.block.1
_backbone.1.1.block.1.avgpool
_backbone.1.1.block.1.fc1
_backbone.1.1.block.1.fc2
_backbone.1.1.block.1.activation
_backbone.1.1.block.1.scale_activation
_backbone.1.1.block.2
_backbone.1.1.block.2.0
_backbone.1.1.block.2.1
_backbone.1.1.stochastic_depth
_backbone.2
_backbone.2.0
_backbone.2.0.block
_backbone.2.0.block.0

In [18]:
ckpt = torch.load("../pretrained_models/model_entire.pth", weights_only=True)

for k, v in ckpt.items():
    print(f"{k}: {v.shape}")

_retina.dog.weight: torch.Size([3, 1, 15, 15])
_backbone.0.0.weight: torch.Size([48, 3, 3, 3])
_backbone.0.1.weight: torch.Size([48])
_backbone.0.1.bias: torch.Size([48])
_backbone.0.1.running_mean: torch.Size([48])
_backbone.0.1.running_var: torch.Size([48])
_backbone.0.1.num_batches_tracked: torch.Size([])
_backbone.1.0.block.0.0.weight: torch.Size([48, 1, 3, 3])
_backbone.1.0.block.0.1.weight: torch.Size([48])
_backbone.1.0.block.0.1.bias: torch.Size([48])
_backbone.1.0.block.0.1.running_mean: torch.Size([48])
_backbone.1.0.block.0.1.running_var: torch.Size([48])
_backbone.1.0.block.0.1.num_batches_tracked: torch.Size([])
_backbone.1.0.block.1.fc1.weight: torch.Size([12, 48, 1, 1])
_backbone.1.0.block.1.fc1.bias: torch.Size([12])
_backbone.1.0.block.1.fc2.weight: torch.Size([48, 12, 1, 1])
_backbone.1.0.block.1.fc2.bias: torch.Size([48])
_backbone.1.0.block.2.0.weight: torch.Size([24, 48, 1, 1])
_backbone.1.0.block.2.1.weight: torch.Size([24])
_backbone.1.0.block.2.1.bias: torch.Siz